In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('building_data/edu.csv')
df

,POI_YM,POI_ID,POI_LON,POI_LAT,CATEGORY,POI_X,POI_Y
0,201901,18504,121.607648,25.055949,中高級校,311308.923926,2.772112e+06
1,201901,16596,121.529918,25.098117,小學,303447.854963,2.776750e+06
2,201901,16263,121.522833,25.108253,小學,302728.898049,2.777870e+06
3,201901,22384,121.501561,25.090958,補習補校,300590.646195,2.775946e+06
4,201901,43092,121.551987,25.100629,幼教,305672.654242,2.777037e+06
...,...,...,...,...,...,...,...
10725,202401,18848,121.568594,25.026015,小學,307382.393942,2.768779e+06
10726,202401,25266,121.574865,25.045367,補習補校,308006.174072,2.770926e+06
10727,202401,34507,121.576631,25.038511,補習補校,308187.609575,2.770167e+06
10728,202401,43159,121.570079,25.041244,幼教,307525.160747,2.770467e+06


In [6]:
print(df['CATEGORY'].unique())

['中高級校' '小學' '補習補校' '幼教' '社區大學' '研究機構' '大專院校' '特教' '外僑學校' '軍警校院']


In [7]:
edu1 = df[df['CATEGORY'].isin(['中高級校', '小學'])]
edu2 = df[df['CATEGORY'].isin(['補習補校', '幼教', '大專院校'])]
edu3 = df[df['CATEGORY'].isin(['特教', '外僑學校', '軍警校院', '社區大學', '研究機構'])]

In [33]:
p_taipei = pd.read_csv('points_taipei.csv')
p_taipei['index'] = range(0, len(p_taipei))
p_taipei

,x1,y1,x2,y2,cx,cy,index
0,296103,2777535,296203,2777635,296153,2777585,0
1,296103,2777635,296203,2777735,296153,2777685,1
2,296103,2777735,296203,2777835,296153,2777785,2
3,296103,2777835,296203,2777935,296153,2777885,3
4,296103,2777935,296203,2778035,296153,2777985,4
...,...,...,...,...,...,...,...
27656,317103,2768935,317203,2769035,317153,2768985,27656
27657,317103,2769035,317203,2769135,317153,2769085,27657
27658,317103,2769135,317203,2769235,317153,2769185,27658
27659,317103,2769235,317203,2769335,317153,2769285,27659


In [34]:
p_new_taipei = pd.read_csv('points_new_taipei.csv')
p_new_taipei['index'] = range(0, len(p_new_taipei))
p_new_taipei

,x1,y1,x2,y2,cx,cy,index
0,278508,2777870,278608,2777970,278558,2777920,0
1,278508,2777970,278608,2778070,278558,2778020,1
2,278508,2778070,278608,2778170,278558,2778120,2
3,278508,2778170,278608,2778270,278558,2778220,3
4,278508,2778370,278608,2778470,278558,2778420,4
...,...,...,...,...,...,...,...
209743,351508,2767770,351608,2767870,351558,2767820,209743
209744,351608,2766970,351708,2767070,351658,2767020,209744
209745,351608,2767070,351708,2767170,351658,2767120,209745
209746,351608,2767270,351708,2767370,351658,2767320,209746


In [10]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [11]:
# 創建一個包含學校位置的GeoDataFrame
edu_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(edu1['POI_X'], edu1['POI_Y'])])

edu1_100 = edu_points1.buffer(100)
edu1_500 = edu_points1.buffer(500)
edu1_1000 = edu_points1.buffer(1000)
edu1_3000 = edu_points1.buffer(3000)

edu1_100_gdf = gpd.GeoDataFrame(geometry=edu1_100, crs=edu_points1.crs)
edu1_500_gdf = gpd.GeoDataFrame(geometry=edu1_500, crs=edu_points1.crs)
edu1_1000_gdf = gpd.GeoDataFrame(geometry=edu1_1000, crs=edu_points1.crs)
edu1_3000_gdf = gpd.GeoDataFrame(geometry=edu1_3000, crs=edu_points1.crs)

In [12]:
# 創建一個包含學校位置的GeoDataFrame2
edu_points2 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(edu2['POI_X'], edu2['POI_Y'])])

edu2_100 = edu_points2.buffer(100)
edu2_1000 = edu_points2.buffer(1000)
edu2_2000 = edu_points2.buffer(2000)

edu2_100_gdf = gpd.GeoDataFrame(geometry=edu2_100, crs=edu_points2.crs)
edu2_1000_gdf = gpd.GeoDataFrame(geometry=edu2_1000, crs=edu_points2.crs)
edu2_2000_gdf = gpd.GeoDataFrame(geometry=edu2_2000, crs=edu_points2.crs)

In [13]:
# 創建一個包含學校位置的GeoDataFrame3
edu_points3 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(edu3['POI_X'], edu3['POI_Y'])])

edu3_100 = edu_points3.buffer(100)
edu3_1000 = edu_points3.buffer(1000)
edu3_2000 = edu_points3.buffer(2000)

edu3_100_gdf = gpd.GeoDataFrame(geometry=edu3_100, crs=edu_points3.crs)
edu3_1000_gdf = gpd.GeoDataFrame(geometry=edu3_1000, crs=edu_points3.crs)
edu3_2000_gdf = gpd.GeoDataFrame(geometry=edu3_2000, crs=edu_points3.crs)

In [14]:
# 將百米方格加載為GeoDataFrame

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)
geometry_gdf_t

,geometry,index
0,"POLYGON ((296103.000 2777535.000, 296203.000 2...",0
1,"POLYGON ((296103.000 2777635.000, 296203.000 2...",1
2,"POLYGON ((296103.000 2777735.000, 296203.000 2...",2
3,"POLYGON ((296103.000 2777835.000, 296203.000 2...",3
4,"POLYGON ((296103.000 2777935.000, 296203.000 2...",4
...,...,...
27656,"POLYGON ((317103.000 2768935.000, 317203.000 2...",27656
27657,"POLYGON ((317103.000 2769035.000, 317203.000 2...",27657
27658,"POLYGON ((317103.000 2769135.000, 317203.000 2...",27658
27659,"POLYGON ((317103.000 2769235.000, 317203.000 2...",27659


In [15]:
def overlay_and_count(geometry_gdf, edu_gdf, prefix):
    intersection = geometry_gdf.overlay(edu_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_edu_counts(geometry_gdf, edu1_100_gdf, edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_100_gdf, edu2_1000_gdf, edu2_2000_gdf, edu3_100_gdf, edu3_1000_gdf, edu3_2000_gdf):
    edu1_100_cnt = overlay_and_count(geometry_gdf, edu1_100_gdf, 'edu1_100')
    edu1_500_cnt = overlay_and_count(geometry_gdf, edu1_500_gdf, 'edu1_500')
    edu1_1000_cnt = overlay_and_count(geometry_gdf, edu1_1000_gdf, 'edu1_1000')
    edu1_3000_cnt = overlay_and_count(geometry_gdf, edu1_3000_gdf, 'edu1_3000')
    edu2_100_cnt = overlay_and_count(geometry_gdf, edu2_100_gdf, 'edu2_100')
    edu2_1000_cnt = overlay_and_count(geometry_gdf, edu2_1000_gdf, 'edu2_1000')
    edu2_2000_cnt = overlay_and_count(geometry_gdf, edu2_2000_gdf, 'edu2_2000')
    edu3_100_cnt = overlay_and_count(geometry_gdf, edu3_100_gdf, 'edu3_100')  
    edu3_1000_cnt = overlay_and_count(geometry_gdf, edu3_1000_gdf, 'edu3_1000')
    edu3_2000_cnt = overlay_and_count(geometry_gdf, edu3_2000_gdf, 'edu3_2000')
    return edu1_100_cnt, edu1_500_cnt, edu1_1000_cnt, edu1_3000_cnt, edu2_100_cnt, edu2_1000_cnt, edu2_2000_cnt, edu3_100_cnt, edu3_1000_cnt, edu3_2000_cnt

# 呼叫函式
edu1_100_cnt_t, edu1_500_cnt_t, edu1_1000_cnt_t, edu1_3000_cnt_t, edu2_100_cnt_t, edu2_1000_cnt_t, edu2_2000_cnt_t, edu3_100_cnt_t, edu3_1000_cnt_t, edu3_2000_cnt_t = calculate_edu_counts(geometry_gdf_t, edu1_100_gdf, edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_100_gdf, edu2_1000_gdf, edu2_2000_gdf, edu3_100_gdf, edu3_1000_gdf, edu3_2000_gdf)
edu1_100_cnt_nt, edu1_500_cnt_nt, edu1_1000_cnt_nt, edu1_3000_cnt_nt, edu2_100_cnt_nt, edu2_1000_cnt_nt, edu2_2000_cnt_nt, edu3_100_cnt_nt, edu3_1000_cnt_nt, edu3_2000_cnt_nt = calculate_edu_counts(geometry_gdf_nt, edu1_100_gdf, edu1_500_gdf, edu1_1000_gdf, edu1_3000_gdf, edu2_100_gdf, edu2_1000_gdf, edu2_2000_gdf, edu3_100_gdf, edu3_1000_gdf, edu3_2000_gdf)


In [35]:
def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['學校加權分數'] = (
        df['edu1_100_cnt'] * 1 +
        df['edu1_500_cnt'] * 0.7 +
        df['edu1_1000_cnt'] * 0.5 +
        df['edu1_3000_cnt'] * 0.3 +
        df['edu2_100_cnt'] * 0.8 +
        df['edu2_1000_cnt'] * 0.5 +
        df['edu2_2000_cnt'] * 0.2 +
        df['edu3_100_cnt'] * 0.6 +
        df['edu3_1000_cnt'] * 0.3 +
        df['edu3_2000_cnt'] * 0.1
    )
    
    return df

# 呼叫函式
result_t = calculate_weighted_score(p_taipei, edu1_100_cnt_t, edu1_500_cnt_t, edu1_1000_cnt_t, edu1_3000_cnt_t, edu2_100_cnt_t, edu2_1000_cnt_t, edu2_2000_cnt_t, edu3_100_cnt_t, edu3_1000_cnt_t, edu3_2000_cnt_t)
result_nt = calculate_weighted_score(p_new_taipei, edu1_100_cnt_nt, edu1_500_cnt_nt, edu1_1000_cnt_nt, edu1_3000_cnt_nt, edu2_100_cnt_nt, edu2_1000_cnt_nt, edu2_2000_cnt_nt, edu3_100_cnt_nt, edu3_1000_cnt_nt, edu3_2000_cnt_nt)

In [38]:
result_t.sort_values('學校加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,edu1_100_cnt,edu1_500_cnt,edu1_1000_cnt,edu1_3000_cnt,edu2_100_cnt,edu2_1000_cnt,edu2_2000_cnt,edu3_100_cnt,edu3_1000_cnt,edu3_2000_cnt,學校加權分數
12694,305603,2769735,305703,2769835,305653,2769785,12694,4.0,10.0,25.0,149.0,33.0,348.0,841.0,0.0,29.0,58.0,451.3
12217,305403,2769735,305503,2769835,305453,2769785,12217,0.0,10.0,30.0,152.0,18.0,362.0,859.0,0.0,31.0,60.0,450.1
12453,305503,2769635,305603,2769735,305553,2769685,12453,0.0,9.0,31.0,151.0,25.0,363.0,822.0,0.0,31.0,62.0,448.5
12454,305503,2769735,305603,2769835,305553,2769785,12454,2.0,10.0,27.0,153.0,21.0,356.0,853.0,0.0,29.0,58.0,448.3
12693,305603,2769635,305703,2769735,305653,2769685,12693,0.0,8.0,26.0,150.0,37.0,353.0,818.0,0.0,30.0,58.0,448.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17099,307203,2787535,307303,2787635,307253,2787585,17099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17100,307203,2787635,307303,2787735,307253,2787685,17100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17101,307203,2787735,307303,2787835,307253,2787785,17101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17102,307203,2787835,307303,2787935,307253,2787885,17102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
result_nt.sort_values('學校加權分數', ascending=False)

,x1,y1,x2,y2,cx,cy,index,edu1_100_cnt,edu1_500_cnt,edu1_1000_cnt,edu1_3000_cnt,edu2_100_cnt,edu2_1000_cnt,edu2_2000_cnt,edu3_100_cnt,edu3_1000_cnt,edu3_2000_cnt,學校加權分數
49172,296708,2766170,296808,2766270,296758,2766220,49172,3.0,7.0,24.0,90.0,42.0,236.0,617.0,0.0,3.0,4.0,323.2
49171,296708,2766070,296808,2766170,296758,2766120,49171,2.0,10.0,22.0,92.0,38.0,230.0,612.0,0.0,1.0,4.0,316.1
80206,302208,2766270,302308,2766370,302258,2766320,80206,3.0,8.0,17.0,126.0,12.0,289.0,527.0,0.0,1.0,7.0,315.4
49761,296808,2766170,296908,2766270,296858,2766220,49761,1.0,7.0,25.0,92.0,23.0,257.0,603.0,0.0,3.0,4.0,314.8
80207,302208,2766370,302308,2766470,302258,2766420,80207,1.0,10.0,14.0,131.0,11.0,283.0,542.0,0.0,1.0,8.0,314.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99045,306408,2792270,306508,2792370,306458,2792320,99045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99044,306408,2792170,306508,2792270,306458,2792220,99044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99043,306408,2792070,306508,2792170,306458,2792120,99043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99042,306408,2791970,306508,2792070,306458,2792020,99042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 寫入csv
result_t.to_csv("taipei_edu.csv", index=False)
result_nt.to_csv("new_taipei_edu.csv", index=False)